# Selection rules

When considering the interaction of light and matter and determining the excitations an electron can undergo by absorption of one photon, the following selection rules are introduced for atoms

\begin{align*}
%&\Delta n  =\textrm{integer} \\ 
&\Delta l  = \pm1\\ 
&\Delta L  =0,\pm1 \\ 
&\Delta S  =0 \\ 
&\Delta J  =0,\pm 1 \quad (J=0 \Rightarrow \Delta J=0)
\end{align*}

for transitions involving changing the electronic state of an atom. Pure electronic excitations can be found in wave lengths of visible light and below. The selection rules can be understood by group theoretical considerations. If the atom (or, more generally, a molecule, for which selection rules can be found in similar manners) interacts with an electric field of incoming photons, electric dipole transitions to higher states are made possible. The probability of a transition from an initial state $\psi_i$ to a final state $\psi_f$ can be calculated as

$$
p_{fi}=\hat{\mu}_{fi}^2= \left| \langle \psi_f | \hat{\mu} | \psi_i \rangle \right|^2
$$

where $\hat{\mu}$ is the electric dipole moment. Again, this integral is by necessity zero if the integrand does not contain the totally symmetric irrep of the point group. In other words, we need to determine the symmetry of the function $\psi_f^{\ast} \hat{\mu} \psi_i$, by taking the direct product of the symmetry species of each function and see if this contains the totally symmetric irrep.

First we must then learn how to determine the symmetry of an electronic state. This is done by identifying all unpaired electrons in the molecule and take the direct product of the corresponding irreps of the molecular orbitals. Note here that the ground state of a closed-shell molecule always belong to the totally symmetric irrep (as all electrons are paired). Remembering that for two functions the symmetry species of the functions much match to acquire the totally symmetric irrep, so this transition is only achievable if

$$
D_{\mu i}(g)=D_{\mu}(g) \otimes D_i(g) \qquad \textrm{has the same symmetry as} \qquad D_f(g)
$$

Consider water with the basis set $(s_H,s_H',s_O,p_x,p_y,p_z)$. Using this it is possible to construct molecular orbitals by use of SALCs such that we obtain three orbitals of $A_1$ symmetry (the totally symmetrical symmetry), two of $B_1$ symmetry and one of $B_2$ symmetry. The ground state is then found to be $(1A_1)^2(1B_1)^2(2A_1)^4(1B_2)^2$ and is, as stated above, of $A_2$ symmetry. The closest antibonding orbital is $(3A_1)$. Further, it can be shown that x-polarized light is of $B_1$ symmetry, y-polarized of $B_2$ symmetry and z-polarized of $A_1$ symmetry.

We are thus prepared to conclude which transitions that can be achieved with which polarizations of incoming light. The first excited state yields the unpaired electrons in $(B_2)(A_1)$, for which the direct product of the two symmetries yields $B_2 \otimes A_1=B_2$. This symmetry matches that of y-polarized light, so this excitation is possible by use of y-polarized light. The second excited state is $(B_2)(B_1)$, with $B_2 \otimes B_1=A_2$. This symmetry does not match that of any polarization of light, and is thus inaccessible. This procedure can be carried out for all possible excitations.

![H-aggregate](../../img/h-aggregate.png) 

In [1]:
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.


In [2]:
dimer_xyz = """6

C        0.67759997    0.00000000    0.00000000
C       -0.67759997    0.00000000    0.00000000
H        1.21655197    0.92414474    0.00000000
H        1.21655197   -0.92414474    0.00000000
H       -1.21655197   -0.92414474    0.00000000
H       -1.21655197    0.92414474    0.00000000"""

molecule = vlx.Molecule.read_xyz_string(dimer_xyz)
basis = vlx.MolecularBasis.read(molecule, "6-31g", ostream=None)

In [3]:
import py3Dmol

viewer = py3Dmol.view(width=400, height=300)

viewer.addModel(dimer_xyz)
viewer.setStyle({"stick": {}, "sphere": {"scale": 0.25}})

# rotate and centralize for a better initial view
viewer.rotate(-90, "x")
viewer.zoomTo()

viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
scf_drv = vlx.ScfRestrictedDriver()

scf_drv.xcfun = "bhandhlyp"
scf_results = scf_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Kohn-Sham                                            
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [5]:
viewer = vlx.OrbitalViewer()
viewer.plot(molecule, basis, scf_drv.mol_orbs)

Output()

Dropdown(description='Orbital:', index=7, options=(('  1 occ=2.000 ene=-10.588', 0), ('  2 occ=2.000 ene=-10.5…

In [6]:
lreig_drv = vlx.LinearResponseEigenSolver()
lreig_drv.nstates = 4

lreig_results = lreig_drv.compute(molecule, basis, scf_results)

                                                                                                                          
                                            Linear Response EigenSolver Setup                                             
                                                                                                                          
                               Number of States                : 4                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-12                                                  
                

In [7]:
print(f"    {'E':8s}{'mu_x':8s}{'mu_y':8s}{'mu_z':8s}{'m_x':8s}{'m_y':8s}{'m_z':8s}{'f':8s}{'R':8s}")
print(72 * "-")

for E, e, m, f, R in zip(
    lreig_results["eigenvalues"],
    lreig_results["electric_transition_dipoles"],
    lreig_results["magnetic_transition_dipoles"],
    lreig_results["oscillator_strengths"],
    lreig_results["rotatory_strengths"],
):
    print(
        f"{E:8.4f}{e[0]:8.4f}{e[1]:8.4f}{e[2]:8.4f}{m[0]:8.4f}{m[1]:8.4f}{m[2]:8.4f}{f:8.4f}{R:8.4f}"
    )

    E       mu_x    mu_y    mu_z    m_x     m_y     m_z     f       R       
------------------------------------------------------------------------
  0.2790 -0.0000 -0.0000  0.0000 -0.0000 -0.9272  0.0000  0.0000  0.0000
  0.3031 -1.6703 -0.0000 -0.0000  0.0000 -0.8399 -0.0000  0.5637 -0.0000
  0.3098 -0.9185 -0.0000  0.0000  0.0000 -0.4570 -0.0000  0.1743 -0.0000
  0.3214  0.0000 -0.0000 -0.0000  0.0000 -0.4909  0.0000  0.0000  0.0000


In [8]:
print(lreig_results["excitation_details"])

[['HOMO     -> LUMO        -0.9789'], ['HOMO-1   -> LUMO         0.9586', 'HOMO     -> LUMO+1       0.2700'], ['HOMO     -> LUMO+1       0.9593', 'HOMO-1   -> LUMO        -0.2752'], ['HOMO-1   -> LUMO+1      -0.9776']]
